### US Immigration Arriving from air Warehouse 

#### Project Summary
The project is aim to create a ware house, where information from immigration arriving in US by air are collect with the origin of the persons, and links with information regading the provenienceand the airports of landing.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

#### Business Scenario/ Project goals
In this project, we will be looking at the immigration data for the united states. More specifically, we're interested on only people arriving trou airports.
We are looking at the following phenomena:

1. the reason of the airival (type of visas and their giustification) ,
2. From which country they are arriving and compare the origiv=cal country caratheristic to beeter identify the reason of the comming.
3. in which airport each natinality prefer to land.





To accomplish this study, we will be using the following datasets:

1. I94 Immigration Data: This data comes from the US National Tourism and Trade Office and includes the contents of the i94 form on entry to the united states. A data dictionary is included in the workspace.
(https://travel.trade.gov/research/reports/i94/historical/2016.html)
    country_dict.csv: table containing country codes used in the dataset, extracted from the data dictionary.
    visa_dict.csv: table containing visa codes used in the dataset, extracted from the data dictionary.

2. Countries Dataset 2020. All about countries - demography, crime index, cost of living etc. Kaggle
(https://www.kaggle.com/dumbgeek/countries-dataset-2020?select=Quality+of+life+index+by+countries+2020.csv)

3. airlines Database kaggle
https://www.kaggle.com/open-flights/airline-database 
    airport_dict.csv: table containing iata codes for teh airports used in the dataset, extracted from the data dictionary.

4. Airport Code Table: This is a simple table of airport codes and corresponding cities.
https://datahub.io/core/airport-codes#data

5. U.S. City Demographic Data: This data comes from OpenSoft. 
(https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)


#### The Architecture

![alt text](https://github.com/EnzoCalogero/capstone_immigration_analysis/blob/master/immages/general_view.png?raw=true )

The whole solution is cloud based of Amazon Web Services (AWS) and based on the follwing steps. 
1. the raw data is saved in a S3 depository.
2. the AWS-EMR/pyspark is processing the immigration dataset reading from s3 depository and saved the post cleanead dataset in the a different folder on the s3 depository.
3. the Apache Airflow pipeline dag is doing the remaing steps of the project:
    - Cleaning the needed dictionaries (country_dict.csv, airport_dict.csv,...)
    - Is loading from the s3 depository, is cleaning  and then save on a different folder th ecleaned dataset Country datasets, City, and Airport datasets
    - All the processed and cleaned dataset are copied on the a pre build Amazon Redshift cluster.
    - finally all the opied data is checked to ensure that all teh previous steps were sucessfull and correct.

![alt text](https://github.com/EnzoCalogero/capstone_immigration_analysis/blob/master/immages/airflow.png?raw=true )


## Overview of the Datasets 


## 1. Immigration Dataset Overview

In [2]:
import pandas as pd

immigration_fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration = pd.read_sas(immigration_fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
immigration.head(2)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1


In [6]:
immigration.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
cicid,6,7,15,16,17,18,19,20,21,22,23,24,27,28,29,30,31,33,34,35
i94yr,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016
i94mon,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
i94cit,692,254,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101
i94res,692,276,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101
i94port,XXX,ATL,WAS,NYC,NYC,NYC,NYC,NYC,NYC,NYC,NYC,TOR,BOS,ATL,ATL,ATL,ATL,HOU,NYC,NYC
arrdate,20573,20551,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545,20545
i94mode,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
i94addr,NaN,AL,MI,MA,MA,MI,NJ,NJ,NY,NY,NY,MO,MA,MA,MA,NJ,NY,TX,CT,CT
depdate,NaN,NaN,20691,20567,20567,20555,20558,20558,20553,20562,20671,20554,20549,20549,20561,20578,20611,20554,NaN,NaN


In [4]:
immigration.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

#### Overview Airport Dataset

In [6]:
airport=pd.read_csv("airport-codes_csv.csv")

In [7]:
airport.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [8]:
airport.shape[0] #number of rows.

55075

In [9]:
airport[airport["iata_code"]=="ATL"]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
26128,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,1026.0,NaN,US,US-GA,Atlanta,KATL,ATL,ATL,"-84.428101, 33.6367"


## CITY Dataset Overview

In [10]:
us_city=pd.read_csv("us-cities-demographics.csv",sep=';')

In [11]:
us_city.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [12]:
us_city[us_city['City']=="Atlanta"]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
59,Atlanta,Georgia,33.8,223960.0,239915.0,463875,18572.0,32016.0,2.15,GA,American Indian and Alaska Native,4606
248,Atlanta,Georgia,33.8,223960.0,239915.0,463875,18572.0,32016.0,2.15,GA,Asian,24268
663,Atlanta,Georgia,33.8,223960.0,239915.0,463875,18572.0,32016.0,2.15,GA,Hispanic or Latino,18653
1899,Atlanta,Georgia,33.8,223960.0,239915.0,463875,18572.0,32016.0,2.15,GA,White,196316
2281,Atlanta,Georgia,33.8,223960.0,239915.0,463875,18572.0,32016.0,2.15,GA,Black or African-American,245367


In [2]:
## Country Datasets Overview

In [14]:
cost=pd.read_csv("country_dataset/Cost of living index by country 2020.csv")

In [15]:
cost.head(5)

,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,Switzerland,122.40,50.25,87.89,120.27,123.01,119.53
1,Norway,101.43,36.15,70.21,91.14,109.28,88.38
2,Iceland,100.48,46.95,74.88,86.89,113.74,79.44
3,Japan,83.35,25.97,55.90,81.82,48.95,87.28
4,Denmark,83.00,31.92,58.57,61.74,100.75,100.88


In [16]:
age=pd.read_csv("country_dataset/Coutries age structure.csv")

In [17]:
age.head(3)

,Country,Age 0 to 14 Years,Age 15 to 64 Years,Age above 65 Years
0,Japan,12.90%,60.10%,27%
1,Italy,13.50%,63.50%,23%
2,Portugal,13.60%,64.90%,22%


In [18]:
crime=pd.read_csv("country_dataset/Crime index by countries 2020.csv")

In [19]:
crime.head(4)

,Country,Crime Index,Safety Index
0,Venezuela,84.49,15.51
1,Papua New Guinea,81.93,18.07
2,South Africa,77.49,22.51
3,Afghanistan,76.23,23.77


In [20]:
health=pd.read_csv("country_dataset/Health care index by countries 2020.csv")

In [21]:
health.head(5)

,Country,Health Care Index,Health Care Exp. Index
0,Taiwan,86.71,159.66
1,South Korea,81.97,149.94
2,Japan,81.14,148.24
3,Denmark,80.00,147.47
4,France,79.99,146.81


In [22]:
property=pd.read_csv("country_dataset/Properties price index by countries 2020.csv")

In [23]:
property.head(8)

,Country,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index
0,Venezuela,133.29,6.22,6.45,16.08,15.49,3025.03,0.03
1,Syria,60.83,2.71,3.00,36.93,33.29,662.83,0.15
2,Hong Kong,47.46,1.77,1.83,56.45,54.74,303.35,0.33
3,Cambodia,42.33,4.78,6.20,20.93,16.13,479.45,0.21
4,Kenya,31.63,2.15,4.07,46.56,24.54,472.19,0.21
5,Sri Lanka,31.14,2.88,2.36,34.76,42.32,478.77,0.21
6,China,29.06,1.71,1.91,58.34,52.38,231.68,0.43
7,Iran,25.11,4.92,5.43,20.33,18.41,515.47,0.19


In [24]:
density=pd.read_csv("country_dataset/Pupulation density by countries.csv")

In [25]:
density.head(2)

,Rank,Country (or dependent territory),Area km2,Area mi2,Population,Density pop./km2,Density pop./mi2,Date,Population source
0,–,Macau,32.90,13,"6,76,100","20,550","53,224","September 30, 2019",Official quarterly estimate
1,1,Monaco,2.02,0.78,"38,300","18,960","49,106","December 31, 2018",Official estimate


In [26]:
Quality=pd.read_csv("country_dataset/Quality of life index by countries 2020.csv")

In [27]:
Quality.head(3)

,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index
0,Denmark,192.67,100.88,74.90,80.00,83.00,7.45,28.85,21.33,81.80
1,Switzerland,192.01,119.53,78.40,72.44,122.40,8.68,29.09,22.39,79.24
2,Finland,190.22,99.93,76.68,75.79,70.29,8.35,29.90,11.55,58.87


## Steps to move from github SAS repository to parquet format

In [28]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [29]:
#write to parquet
df_spark.write.parquet("sas_data")


In [30]:
# Test is working the just created parquet
df_spark=spark.read.parquet("sas_data")
df_spark.head(1)

[Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')]

### Step 2: Explore and Assess the Data
#### Explore the Data &  Cleaning Steps

Document steps necessary to clean the data

[jupiter Notepad with full steps for cleaning Dimensions datasets](https://github.com/EnzoCalogero/capstone_immigration_analysis/blob/master/cleaning%20Datasets/cleaning_dimensions.ipynb)

[jupiter Notepad with full steps for cleaning immigration datasets](https://github.com/EnzoCalogero/capstone_immigration_analysis/blob/master/cleaning%20Datasets/cleaning%20Immigration.ipynb)

### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model

The immigration dataset is the origin of the center of our model. As this represent the facts we are analysing - U.S visitors from the world arriving by airplan, this was transformed to the fact table IMMIGRATION as represented in the schema below. 

![alt text](https://github.com/EnzoCalogero/capstone_immigration_analysis/blob/master/immages/capstone.png?raw=true )

https://dbdiagram.io/d/5eda66119ea313663b3a1b45



#### Dimension Tables

The  dataset **"Countries Dataset 2020"** is composed of different csv files, each one focusing on different countries metrics.
From each csv file is extracted an independent table, all the created tables are linked with res_country (country of residence) and cit_country (citadinace).

They are:
1. *Country Crime*
2. *Country life Quality*
3. *Country Health*
4. *Country Cost*
5. *Country Age*
6. *Country Property Cost*
7. *Country Pupolation*
    
  

The dataset **airlines Database kaggle** is loaded on the table *airlines* and is linked the column *IATA* with the column *airline*in the *immigration* table 

The dataset **Airport Code Table** is loaded on the table *airports* and is linked the column *IATA* with the column *airport*in the *immigration* table  

The dataset **U.S. City Demographic Data (OpenSoft)** is loaded on the table *cities* and is linked the column *city* with the column *municipality*in the *airports* table  




#### 3.2 Mapping Out Data Pipelines
The steps on which th eproject is based are the fillowing:

1. The project is expecting all the raw data in a s3 repository:
    * all the dimensional dataset csv files are copied on a folder (/raw_data) on the s3 depository.
    * The immigration dataset is extracted from the SAS format into the parquet format and saved as well on the s3 depository.
    (code used ......)

2.  an EMR server is started to process the PySpark code to clean and transform the immigration dataset.
    (code....)
    * The main steps is filter out only the arrival from air;
    * Considering only the relevant variable;
    * Replace in the two columns res_country and cit_country the country code with the actually country name;    
    * Replace in the columns visa the visa code with the actually visa name;
    * save the cleaned dataset on the S3 depository in parquet format.

3.     



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.